# **Data Wrangling**
---

> The purpose of this notebook is to modify the bus data provided by the STM and the historical weather data obtained from the Government of Canada site.  
> Running this script is not necessary as the modified data is available upon download/cloning of the repository.

In [1]:
import os
import pandas as pd
import numpy as np
import time
import urllib.request
import glob
from datetime import datetime, timedelta

## STM Data

> This data was provided by the STM under the terms of the *Act respecting access to documents held by public bodies and the protection of personal information (R.S.Q., chapter A-2.1)*.

In [6]:
# Merge STM files
STM_1_df = pd.read_csv('../Data/Transit data/STM_Data_2021_2022.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
STM_2_df = pd.read_csv('../Data/Transit data/STM_Data_2023.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
STM_1_df = STM_1_df.dropna(how='all')
STM_df = pd.concat([STM_1_df, STM_2_df], ignore_index=True)

# Remove duplicate rows
original_df = STM_df.copy()
STM_df = STM_df.drop_duplicates()
removed_rows = original_df[~original_df.index.isin(STM_df.index)]

STM_df.head()

,date,ligne,dir,id_voy,dep_pl,dep_rl,arr_pl,arr_rl
0,2021/10/05,100.0,Est,"35,081,296",00:10:00,00:10:07,00:45:00,00:44:54
1,2021/10/05,100.0,Est,"35,080,889",00:42:00,00:42:20,01:17:00,01:14:05
2,2021/10/05,100.0,Est,"35,083,075",01:14:00,01:13:51,01:49:00,01:44:43
3,2021/10/05,100.0,Est,"35,080,237",05:06:00,05:06:21,05:41:00,05:39:21
4,2021/10/05,100.0,Est,"35,081,071",05:39:00,05:41:07,06:14:00,06:13:06


In [7]:
# Date and time formatting
# The date and time columns are given in HH:MM:SS as strings, we convert them to datetime objects with their assigned time to find delays later

# Change data format from YYYY/MM/DD to YYYY-MM-DD and assign to hour columns, HH:MM:SS is changed to YYYY-MM-DD HH:MM:SS
try:
    STM_df['date'] = STM_df['date'].str.replace('/', '-')
    STM_df['date'] = pd.to_datetime(STM_df['date'],errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Date: {e}")

try:
    for col in ['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']:
        STM_df[col] = pd.to_datetime(STM_df[col], format='%H:%M:%S').dt.time
        STM_df[col] = STM_df[col].astype(str)
        STM_df[col] = STM_df['date'].astype(str) + ' ' + STM_df[col]
        STM_df[col] = pd.to_datetime(STM_df[col], errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Time: {e}")

# Some datapoints have a trip that was scheduled for a day but actually occurred the next day, this is compensated for as follows:
#   If the time in pl column is after 23:00 and in rl column is before 1:00, add 1 day to the date in rl column
#   -> This is to account for arrivals that were scheduled for late night but actually occurred early morning
#   If the time in pl column is before 1:00 and in rl column is after 23:00, substract 1 day to the date in rl column
#   -> This is to account for arrivals that were scheduled for early morning but actually occurred late the previous night

def adjust_dates(row):
    if row['dep_pl'].hour >= 23 and row['dep_rl'].hour < 1:
        row['dep_rl'] += pd.Timedelta(days=1)
    elif row['dep_pl'].hour < 1 and row['dep_rl'].hour >= 23:
        row['dep_rl'] -= pd.Timedelta(days=1)

    if row['arr_pl'].hour >= 23 and row['arr_rl'].hour < 1:
        row['arr_rl'] += pd.Timedelta(days=1)
    elif row['arr_pl'].hour < 1 and row['arr_rl'].hour >= 23:
        row['arr_rl'] -= pd.Timedelta(days=1)
    
    return row

STM_df = STM_df.apply(adjust_dates, axis=1)

# Slice the time to only keep HH:MM
try:
    for col in ['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']:
        STM_df[col] = STM_df[col].dt.strftime('%H:%M')
except Exception as e:
    print(f"An error occurred while formatting the Time: {e}")

STM_df.head()

,date,ligne,dir,id_voy,dep_pl,dep_rl,arr_pl,arr_rl
0,2021-10-05,100.0,Est,"35,081,296",00:10,00:10,00:45,00:44
1,2021-10-05,100.0,Est,"35,080,889",00:42,00:42,01:17,01:14
2,2021-10-05,100.0,Est,"35,083,075",01:14,01:13,01:49,01:44
3,2021-10-05,100.0,Est,"35,080,237",05:06,05:06,05:41,05:39
4,2021-10-05,100.0,Est,"35,081,071",05:39,05:41,06:14,06:13


In [8]:
#Indexing

#Reset the index before adding the new 'id' column
STM_df = STM_df.reset_index(drop=True)

#add an index with the name 'id' for which the numbers start with a period
STM_df['id'] = STM_df.index + 1
STM_df['id'] = STM_df['id'].apply(lambda x: '.' + str(x))

#move index to the first column
cols = list(STM_df.columns)
cols = [cols[-1]] + cols[:-1]
STM_df = STM_df[cols]

STM_df.head()


,id,date,ligne,dir,id_voy,dep_pl,dep_rl,arr_pl,arr_rl
0,.1,2021-10-05,100.0,Est,"35,081,296",00:10,00:10,00:45,00:44
1,.2,2021-10-05,100.0,Est,"35,080,889",00:42,00:42,01:17,01:14
2,.3,2021-10-05,100.0,Est,"35,083,075",01:14,01:13,01:49,01:44
3,.4,2021-10-05,100.0,Est,"35,080,237",05:06,05:06,05:41,05:39
4,.5,2021-10-05,100.0,Est,"35,081,071",05:39,05:41,06:14,06:13


In [ ]:
# Uncomment to save the dataframe as a csv file
# STM_df.to_csv('../Data/Transit data/STM_Data.csv', index=False)

## Weather Data

>The following cells retrieve the weather data from the [Environment Canada website](https://climate.weather.gc.ca/historical_data/search_historic_data_e.html)

In [ ]:
# Select search parameters
yi = 2021 # Select start year
yf = 2023 + 1 # Select end year
station = 30165 # Select station ID (Montreal-Trudeau Airport is 30165)


#Retrieve daily weather data from the Government of Canada website and store in Individual Files folder
for year in range(yi, yf):
        url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station) + '&Year=' + str(year) + '&Month=1&Day=14&timeframe=2&submit=Download+Data'
        filename = os.path.join('../Data/Weather Data/Individual Files', 'daily_montreal_weather_' + str(year) + '.csv')
        try: 
            urllib.request.urlretrieve(url, filename)
        except Exception as e:
            print(f"An error occurred while downloading data for year {year} and month {month}: {e}")
        time.sleep(5)

#Retrieve hourly weather data from the Government of Canada website and store in Individual Files folder
for year in range(yi, yf):
   
    if year == 2021:
        mi = 10
        mf = 12 + 1
    elif year == 2023:
        mi = 1
        mf = 9 + 1
    else:
        mi = 1
        mf = 12 + 1
    for month in range(mi, mf):
        if month < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&time=UTC&stationID=' + str(station) + '&Year=' + str(year) + '&Month=' + str(month) + '&Day=14&timeframe=1&submit=Download+Data'
        filename = os.path.join('../Data/Weather Data/Individual Files', 'hourly_montreal_weather_' + str(year) + '_' + str(month) + '.csv')
        try:
            urllib.request.urlretrieve(url, filename)
        except Exception as e:
            print(f"An error occurred while downloading data for year {year} and month {month}: {e}")
        time.sleep(5)

# merge all csv files into their corresponing dataframes and store them in the Weather Data folder
all_dailyw = glob.glob(os.path.join('../Data/Weather Data/Individual Files', "daily_montreal_weather_*.csv")) 
all_hourlyw = glob.glob(os.path.join('../Data/Weather Data/Individual Files', "hourly_montreal_weather_*.csv"))

df_all_dailyw = (pd.read_csv(f) for f in all_dailyw)
df_dailyw = pd.concat(df_all_dailyw, ignore_index=True)
df_all_hourlyw = (pd.read_csv(f) for f in all_hourlyw)
df_hourlyw = pd.concat(df_all_hourlyw, ignore_index=True)

df_dailyw.to_csv('../Data/Weather Data/daily_montreal_weather.csv', index=False)
df_hourlyw.to_csv('../Data/Weather Data/hourly_montreal_weather.csv', index=False)

# delete all individual csv files
files = glob.glob('../Data/Weather Data/Individual Files/*')
for f in files:
    os.remove(f)

1. Daily data for snow

In [14]:
# Read back the daily weather data to perform formatting
# Since snow on ground is only available for daily data, the presence of snow is determined by the daily data, shown as Y/N
# The daily data is later applied to the hourly data (see master data below)

df_dailyw = pd.read_csv('../Data/Weather Data/daily_montreal_weather.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
df_dailyw = df_dailyw.dropna(axis=1, how='all')
df_dailyw = df_dailyw.rename(columns={'Date/Time': 'date', 'Snow on Grnd (cm)': 'snow'})

In [15]:
# Date and time formatting
# Change date format and remove rows with dates that are not in the STM data
try:
    df_dailyw['date'] = pd.to_datetime(df_dailyw['date'],errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Date: {e}")
    
df_dailyw = df_dailyw[df_dailyw['date'].isin(STM_df['date'])]

In [ ]:
# Place the date and snow columns first and remove the rest

cols = df_dailyw.columns.tolist()  
fifth_col = cols[4]
fifth_to_last_col = cols[-5]
cols = [col for col in cols if col not in {cols[4], fifth_to_last_col}]
cols = [fifth_col, fifth_to_last_col]
df_dailyw = df_dailyw[cols]

# Place a value of 0 for snow if NaN
df_dailyw['snow'] = df_dailyw['snow'].fillna(0)

# Place a value of Y for snow if >0 and N if 0 or Nan in a new column
df_dailyw['snow_yn'] = np.where(df_dailyw['snow'] > 0, 'Y', 'N')

In [16]:
df_dailyw.head()

,date,snow,snow_yn
0,2021-10-05,0.0,N
1,2021-10-06,0.0,N
2,2021-10-07,0.0,N
3,2021-10-08,0.0,N
4,2021-10-09,0.0,N


In [ ]:
# Uncomment to save the dataframe as a csv file
#df_dailyw.to_csv('../Data/Weather Data/daily_montreal_weather.csv', index=False)

2) Hourly data for precipitation

In [ ]:
# Read back the hourly weather data to perform formatting
# The hourly data is formatted as UTC, the time is converted to EST
df_hourlyw = pd.read_csv('../Data/Weather Data/hourly_montreal_weather.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
df_hourlyw = df_hourlyw.dropna(axis=1, how='all')
df_hourlyw = df_hourlyw.rename(columns={'Date/Time (UTC)': 'date', 'Time (UTC)': 'time', 'Temp (°C)' : 'temp', 'Precip. Amount (mm)' : 'precip'})

In [ ]:
# Change format of date columns to datetime, change from UTC to EST taking into account daylight savings time
# Change time column to reflect the time in the date column
try:
    df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'],errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Date: {e}")

df_hourlyw['date'] = df_hourlyw['date'].dt.tz_localize('UTC').dt.tz_convert('America/Montreal')
df_hourlyw['time'] = df_hourlyw['date'].dt.strftime('%H:%M')
df_hourlyw['date'] = df_hourlyw['date'].dt.date

# Change date format and remove rows with dates that are not in the STM data
try:
    df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'],errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Date: {e}")
    
df_hourlyw = df_hourlyw[df_hourlyw['date'].isin(STM_df['date'])]

# Place the date, time, temp and precip columns first and remove the rest
cols = df_hourlyw.columns.tolist()
date_col = cols[4]
time_col = cols[8]
temp_col = cols[9]
precip_col = cols[12]
cols = [col for col in cols if col not in {date_col, time_col, temp_col, precip_col}]
cols = [date_col, time_col, temp_col, precip_col]
df_hourlyw = df_hourlyw[cols]

In [ ]:
# Uncomment to save the dataframe as a csv file
#df_hourlyw.to_csv('../Data/Weather Data/hourly_montreal_weather.csv', index=False)

## Master data

> The master data used for this project includes the following:
> - Bus data
>   - Date*
>   - Line
>   - Start time
>   - Start delay
>   - End time*
> - Weather data
>   - Temperature at start 
>   - Rain amount
>   - Snow amount
>   - Total snow*
>
> These variables will be used to predict the end delay.  
> *: These variables are included in the master data for later use, but they are not used as predictors

1. Bus data (date, line, start_time, end_time)

In [17]:
# Create a new dataframe for the master data with columns from the STM data
STM_df = pd.read_csv('../Data/Transit data/STM_Data.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str,8:str})
df_master = STM_df[['id','date', 'ligne', 'dep_rl', 'arr_rl']]
df_hourlyw = pd.read_csv('../Data/Weather Data/hourly_montreal_weather.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
df_dailyw = pd.read_csv('../Data/Weather Data/daily_montreal_weather.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})

df_master = df_master.rename(columns={'dep_rl': 'start_time', 'arr_rl': 'end_time', 'ligne': 'line'})
df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'], errors='coerce')
df_dailyw['date'] = pd.to_datetime(df_dailyw['date'], errors='coerce')

# Cut the date from the start_time column and keep only the time as HH:MM
for col in ['start_time', 'end_time']:
    df_master[col] = df_master[col].astype(str)
    df_master[col] = df_master[col].str.slice(start=11, stop=16)
    df_master[col] = pd.to_datetime(df_master[col], format='%H:%M', errors='coerce').dt.time

# Delete nan values in the start_time column
df_master = df_master.dropna(subset=['start_time'])

df_master.head()

,id,date,line,start_time,end_time
0,.1,2021-10-05,100.0,00:10:00,00:44:00
1,.2,2021-10-05,100.0,00:42:00,01:14:00
2,.3,2021-10-05,100.0,01:13:00,01:44:00
3,.4,2021-10-05,100.0,05:06:00,05:39:00
4,.5,2021-10-05,100.0,05:41:00,06:13:00


2. Temperature at start (temp_start)

In [18]:
# Add temp_start column to the master data, using the hourly weather data which matches the date and hour of the start time
# Must create an hour columm in the master data to match the time column in the weather data (HH:MM)
# Match atch the start hour with the next hour in the weather data since the weather time is the end of the hour

df_master['hour'] = df_master['start_time'].astype(str)
df_master['hour'] = df_master['hour'].str.slice(start=0, stop=2)
df_master['hour'] = df_master['hour'].astype(int)
df_master['hour'] = df_master['hour'] + 1
df_master['hour'] = df_master['hour'].astype(str)
df_master['hour'] = df_master['hour'].str.pad(width=2, side='left', fillchar='0')
df_master['hour'] = df_master['hour'] + ':00'

# Rename the time column in hourly data to hour
df_hourlyw = df_hourlyw.rename(columns={'time': 'hour'})

df_master['date'] = pd.to_datetime(df_master['date'])
df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'])

df_master = pd.merge(df_master, df_hourlyw, how='left', on=['date', 'hour'])
df_master = df_master.rename(columns={'temp': 'temp_start'})
df_master = df_master.drop(columns=['hour'])

df_master.head()

,id,date,line,start_time,end_time,temp_start,precip
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0


3. Precipitation (snow_amt, rain_amt)

In [19]:
# Add rain_amt and snow_amt columns to the master data, using the daily weather data which matches the date
# For the snow_amt column, if the snow_yn column is Y, then take the value of the precip column, otherwise set to 0
# For the rain_amt column, if the snow_yn column is N, then take the value of the precip column, otherwise set to 0

df_master = pd.merge(df_master, df_dailyw, how='left', left_on=['date'], right_on=['date'])
df_master = df_master.rename(columns={'precip': 'rain_amt', 'snow': 'total_snow'})
df_master['snow_amt'] = np.where(df_master['snow_yn'] == 'Y', df_master['rain_amt'], 0)
df_master['rain_amt'] = np.where(df_master['snow_yn'] == 'N', df_master['rain_amt'], 0)
df_master = df_master.drop(columns=['snow_yn',])

df_master.head()

,id,date,line,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0,0.0,0
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0,0.0,0
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0,0.0,0
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0,0.0,0
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0,0.0,0


4. Delays (delay_start, delay_end)

In [20]:
# Import dep_pl, dep_rl, arr_pl, arr_rl columns from the stm data to the master data
# Cut the seconds off from the time columns
df_master = pd.merge(df_master, STM_df[['id', 'dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']], how='left', left_on=['id'], right_on=['id'])

for col in ['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']:
    df_master[col] = df_master[col].astype(str)
    df_master[col] = df_master[col].str.slice(start=0, stop=16)
    df_master[col] = df_master[col] + ':00'
    df_master[col] = pd.to_datetime(df_master[col], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Find the delays in minutes between the planned and real departure times in minutes
df_master['delay_start'] = (df_master['dep_rl'] - df_master['dep_pl']).dt.total_seconds() / 60
df_master['delay_start'] = df_master['delay_start'].astype(int)

df_master['delay_end'] = (df_master['arr_rl'] - df_master['arr_pl']).dt.total_seconds() / 60
df_master['delay_end'] = df_master['delay_end'].astype(int)

df_master = df_master.drop(columns=['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl'])

df_master.head()


,id,date,line,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt,delay_start,delay_end
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0,0.0,0,0,-1
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0,0.0,0,0,-3
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0,0.0,0,-1,-5
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0,0.0,0,0,-2
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0,0.0,0,2,-1


In [21]:
# Verify there are not many delays over 10 hours

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(df_master.loc[abs(df_master['delay_start']) > 600])
print(df_master.loc[abs(df_master['delay_end']) > 600])

             id       date   line start_time  end_time temp_start rain_amt  \
14637    .14638 2021-10-20  139.0   07:50:00  22:32:00        6.9      0.0   
39737    .39719 2021-11-15  460.0   07:45:00  20:29:00        3.0      0.2   
167836  .167821 2022-03-20  139.0   17:11:00  00:51:00        4.2        0   
211850  .211835 2022-05-03   67.0   22:35:00  01:06:00       14.2      0.0   
271315  .271300 2022-07-06  460.0   06:16:00  18:12:00       16.8      0.0   
314365  .314350 2022-08-24   80.0   23:08:00  01:32:00        NaN      NaN   
361995  .361980 2022-10-15  121.0   21:42:00  00:37:00       12.8      0.0   
538933  .538897 2023-04-08   80.0   23:50:00  01:43:00        NaN      NaN   

        total_snow snow_amt  delay_start  delay_end  
14637          0.0        0         -834          2  
39737          0.0        0         -706         -2  
167836         2.0      0.8         1018         -1  
211850         0.0        0         1326          0  
271315         0.0        0

5. Organization

In [22]:
cols = df_master.columns.tolist()
id_col = cols[0]
date_col = cols[1]
line_col = cols[2]
delay_start_col = cols[9]
cols = [col for col in cols if col not in {id_col,date_col, line_col, delay_start_col}]

cols = [id_col,date_col, line_col, delay_start_col] + cols
df_master = df_master[cols]

df_master.head()


,id,date,line,delay_start,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt,delay_end
0,.1,2021-10-05,100.0,0,00:10:00,00:44:00,15.1,0.0,0.0,0,-1
1,.2,2021-10-05,100.0,0,00:42:00,01:14:00,15.1,0.0,0.0,0,-3
2,.3,2021-10-05,100.0,-1,01:13:00,01:44:00,13.7,0.0,0.0,0,-5
3,.4,2021-10-05,100.0,0,05:06:00,05:39:00,11.8,0.0,0.0,0,-2
4,.5,2021-10-05,100.0,2,05:41:00,06:13:00,11.8,0.0,0.0,0,-1


In [23]:
# Show unique line values
print(df_master['line'].unique())
print(STM_df['ligne'].unique())

['100.0' '121.0' '139.0' '439.0' '460.0' '467.0' '480.0' '67.0' '80.0']
['100.0' '121.0' '139.0' '439.0' '460.0' '467.0' '480.0' '67.0' '80.0']


In [ ]:
#Seems like the STM did not provide data for the 193 line ._.

In [ ]:
# Uncomment to save the dataframe as a csv file
#df_master.to_csv('../Data/master_data.csv', index=False)